In [ ]:
import torch
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm_notebook as tqdm
from rsna19.configs.second_level import Config
from sklearn.metrics import log_loss
import pandas as pd
from scipy.signal import windows

In [ ]:
train_folds = [0, 1, 2, 3]
val_folds = [4]

train_x = torch.cat([torch.tensor(np.load(f'/kolos/m2/ct/models/classification/rsna-cache/v07/cache/fold{f}/x.npy'), dtype=torch.float32) for f in train_folds], dim=0)
train_y = torch.cat([torch.tensor(np.load(f'/kolos/m2/ct/models/classification/rsna-cache/v07/cache/fold{f}/y.npy'), dtype=torch.float32) for f in train_folds], dim=0)
val_x = torch.cat([torch.tensor(np.load(f'/kolos/m2/ct/models/classification/rsna-cache/v07/cache/fold{f}/x.npy'), dtype=torch.float32) for f in val_folds], dim=0)
val_y = torch.cat([torch.tensor(np.load(f'/kolos/m2/ct/models/classification/rsna-cache/v07/cache/fold{f}/y.npy'), dtype=torch.float32) for f in val_folds], dim=0)

n_models = 5
class_weights = torch.tensor([1, 1, 1, 1, 1, 2], dtype=torch.float32) * 6 / 7
loss_fn = F.binary_cross_entropy

In [ ]:
# undo sigmoid

# train_x[train_x > 0] = torch.log(train_x[train_x > 0] / (1-train_x[train_x > 0]))
# val_x[val_x > 0] = torch.log(val_x[val_x > 0] / (1-val_x[val_x > 0]))
# loss_fn = F.binary_cross_entropy_with_logits

In [ ]:
print('train')
preds = []
for model_id in range(n_models):
    preds.append(train_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], train_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, train_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

print('\nval')
preds = []
for model_id in range(n_models):
    preds.append(val_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], val_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, val_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

In [ ]:
# train on middle slice only

# train_x_1slice = []
# for model_id in range(n_models):
#     train_x_1slice.append(train_x[:, model_id*30+12:model_id*30+18])
# train_x_1slice = torch.cat(train_x_1slice, dim=1)

# val_x_1slice = []
# for model_id in range(n_models):
#     val_x_1slice.append(val_x[:, model_id*30+12:model_id*30+18])
# val_x_1slice = torch.cat(val_x_1slice, dim=1)

# train_x = train_x_1slice
# val_x = val_x_1slice

In [ ]:
hidden = 128
features_out = 6

# model = torch.nn.Sequential(
#         torch.nn.Linear(train_x.shape[1], features_out)
# )

model = torch.nn.Sequential(
        torch.nn.Linear(train_x.shape[1], hidden),
        torch.nn.ReLU(),
#         torch.nn.Dropout(0.2),
        torch.nn.Linear(hidden, features_out),
)

train_x = train_x.cuda()
train_y = train_y.cuda()
val_x = val_x.cuda()
val_y = val_y.cuda()
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), 0.001)
val_log_loss = 0
class_weights = class_weights.cuda()

for i in tqdm(range(5000)):
    optimizer.zero_grad()

    y_hat = model(train_x)
    loss = F.binary_cross_entropy_with_logits(y_hat, train_y, weight=class_weights)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        model.eval()
        val_y_hat = model(val_x)
        val_loss = F.binary_cross_entropy_with_logits(val_y_hat, val_y, weight=class_weights)
        model.train()
        
        print(f'{i:04d}: train: {loss.item():.04f}, val: {val_loss.item():.04f}')